In [40]:
# Extrahiere Tabelle / Philosophennamen aus:
# https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte
#
# Mozilla Firefox shortcuts
# 
# 
# Jupyter Shortcuts: 'H'
#
# https://www.tutorialspoint.com/python_web_scraping/python_modules_for_web_scraping.htm
# https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-2/
# https://oxylabs.io/blog/python-web-scraping
# https://www.freecodecamp.org/news/web-scraping-python-tutorial-how-to-scrape-data-from-a-website/
# https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/
#
# https://www.geeksforgeeks.org/parsing-tables-and-xml-with-beautifulsoup/
# https://towardsdatascience.com/a-guide-to-scraping-html-tables-with-pandas-and-beautifulsoup-7fc24c331cf7
#

In [41]:
import sys
import requests
import csv
from bs4 import BeautifulSoup


In [42]:
web_page_address = "https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte"
print (web_page_address)

https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte


In [43]:
web_page = requests.get(web_page_address)
print ("web_page = \t\t", web_page)
print ("web_page.status_code = \t", web_page.status_code)


web_page = 		 <Response [200]>
web_page.status_code = 	 200


In [44]:
# print ("web_page.content = \t", web_page.content)

In [45]:
soup = BeautifulSoup(web_page.content, 'html.parser')
# print(soup.prettify())

In [46]:
bs4_table = soup.find('table', class_='wikitable toptextcells')
# print (type(bs4_table))

In [47]:
def csv_event_file_header_get():
    event_header = ("start", "end", "text", "progress", "fuzzy", "locked", "ends_today", "category", "description", "hyperlink", "alert", "icon", "default_color", "milestone")
    return event_header

In [48]:
def csv_event_file_open():
    fn = 'csv_tst.csv'
    # with open(sys.argv[1], 'wt') as f:
    with open(fn, 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(csv_event_file_header_get())
        for i in range(3):
            row = (
                i + 1,
                chr(ord('a') + i),
                '08/{:02d}/07'.format(i + 1),
                i,
            )
            writer.writerow(row )

    # print(open(sys.argv[1], 'rt').read())    pass
    print (' -------------------')
    print(open(fn, 'rt').read())

In [49]:
csv_event_file_open()
def csv_event_make():
    pass

 -------------------
start,end,text,progress,fuzzy,locked,ends_today,category,description,hyperlink,alert,icon,default_color,milestone

1,a,08/01/07,0

2,b,08/02/07,1

3,c,08/03/07,2




In [72]:
row_cnt = 0

lo_philosopher  = []     # lo == list of

class Philosopher:    # el == element
    def __init__(self):
        self.name        = ''     # Name
        self.lifespan    = ''     # Lifespan
        self.lo_works    = None   # Books etc
        self.lo_theories = None   # Theories
        self.lo_history  = None   # Events
        self.cnt         = 0      # Number

philosoph_cnt = 0

for bs4_row in bs4_table.find_all('tr'):
    # print (type(bs4_row))   # type == <class 'bs4.element.Tag'>
    bs4_row_text = BeautifulSoup(bs4_row.get_text())  # get_text() == 'converts' type to string, sort of
    row_cnt += 1
    # https://stackoverflow.com/questions/13853025/is-there-a-way-in-beautiful-soup-to-count-the-number-of-tags-in-a-html-page
    td_columns = len (bs4_row.find_all('td'))
    # print ("td_columns = ", td_columns)

    philosopher = Philosopher()


    if (td_columns == 1):    # Headline of span
        span_mw_headline = bs4_row.find_all('span', class_='mw-headline')
        for smh in span_mw_headline:
            print('span_mw_headline >> ', smh.string)

    elif (td_columns == 4) :
        philosoph_lifespan      = None
        philosoph_full_name     = None

        for td_cnt, td_item in enumerate (bs4_row.find_all('td')):
            if td_item.string:
                td_item_string = td_item.string.replace("\n", " ")
            elif td_item.text:
                td_item_string = td_item.text
            else:
                td_item_string = ''
            print('td_cnt: ', td_cnt, ',   td_item: ', td_item_string)
            pass

        for item_cnt, item in enumerate (bs4_row):
            if item_cnt == 1:
                # print(row_cnt, 'cnt == 1: ', str(item).replace("\n", " "))
                philosoph_lifespan = item.string

            elif item_cnt == 3:
                # print(row_cnt, 'cnt == 3: ', str(item).replace("\n", " "))
                # print('philosoph_full_name = >', philosoph_full_name, '<')
                # philosoph_full_name = item.a 
                # if philosoph_full_name:
                name = '?'
                if item.a:
                    philosoph_full_name = item.a.string                 # Name
                    if ( ' von ' in philosoph_full_name) or \
                       ( ' der ' in philosoph_full_name) :
                        # print (row_cnt, philosoph_full_name, end = '')
                        name = philosoph_full_name
                    else:
                        # print(philosoph_full_name.split(' ')[-1], end = '')         # Nachname
                        name = philosoph_full_name.split(' ')[-1]
                        if philosoph_full_name.split(' ')[:-1]:
                            # print(', ', end = '')
                            name = name + ', '
                        # [print (ele, end = ' ') for ele in philosoph_full_name.split(' ')[:-1]]  # alle Vornamen
                        for ele in philosoph_full_name.split(' ')[:-1]:
                            name = name + ele + ' '
                    # print()

                if philosoph_full_name:
                    #print(row_cnt, '### ', philosoph_full_name)
                    #print('--- ', philosoph_lifespan)
                    philosoph_cnt += 1
                    philosopher.name = name
                    try:
                        philosoph_lifespan = philosoph_lifespan.string.replace("\n", " ")
                    except:
                        philosoph_lifespan = '???? '
                    
                    # line_new = '{:>12}  {:>12}  '.format(philosoph_lifespan, name)
                    line_new = '{:>35}      {}  '.format(philosoph_lifespan, name)
                    print (philosoph_cnt, ': ',line_new)
                    
                    philosopher.lifespan = philosoph_lifespan

    lo_philosopher.append(philosopher)


td_columns =  1
span_mw_headline >>  Antike
td_columns =  1
span_mw_headline >>  Vorsokratiker 600–400 v. Chr.
span_mw_headline >>  Ionische Naturphilosophie
td_columns =  0
td_columns =  4
td_cnt:  0 ,   td_item:  um 624–546 
td_cnt:  1 ,   td_item:  Thales von Milet

td_cnt:  2 ,   td_item:  
Sagte Sonnenfinsternis voraus
Wasser als Urstoff

td_cnt:  3 ,   td_item:  
um 621 Gesetzgebung Drakons
594 Reform Solons

1 :                          um 624–546       Thales von Milet  
td_columns =  4
td_cnt:  0 ,   td_item:  um 610–547 
td_cnt:  1 ,   td_item:  Anaximander

td_cnt:  2 ,   td_item:  
Erster Entwurf einer Kosmogonie
Urstoff ist das räumlich und zeitlich Unbeschränkte (Apeiron): bereits Anaximander verwendete also einen über die Erfahrung hinausgehenden metaphysischen Erklärungsbegriff

td_cnt:  3 ,   td_item:   
2 :                          um 610–547       Anaximander  
td_columns =  4
td_cnt:  0 ,   td_item:  um 585–525 
td_cnt:  1 ,   td_item:  Anaximenes

td_cnt:  2 ,   td